<a href="https://colab.research.google.com/github/agi2019/ppi-gci/blob/main/tutorials/02%20-%20model%20calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Model calibration</center>

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, <a href="https://twitter.com/guerrero_oa">@guerrero_oa</a>)

In this tutorial I will calibrate the free parameters of PPI's model. First, I will load all the data that I have prepared in the previous tutorials. Then, I extract the relevant information and put it in adequate data structures. Finally, I run the calibration function and save the results with the parameter values.

## Importing Python libraries to manipulate data

In [1]:
import pandas as pd
import numpy as np

## Importing PPI functions

In this tutorial, I will import the PPI source code directly from its repository. This means that I will place a request to GitHub, download the `policy_priority_inference.py` file, and copy it locally into the folder where these tutorials are saved. Then, I will import PPI. This approach is useful if you want to run this tutorial in a cloud computing service.

An alternative would be to manually copy the `policy_priority_inference.py` file into the folder where this tutorial is located.

In [2]:
import requests # the Python library that helps placing requests to websites
url = 'https://raw.githubusercontent.com/agi2019/ppi-gci/main/source_code/policy_priority_inference.py'
r = requests.get(url)
with open('policy_priority_inference.py', 'w') as f:
    f.write(r.text)
import policy_priority_inference as ppi

## Load data

### Indicators

In [3]:
df_indis = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_indicators.csv')

N = len(df_indis) # number of indicators
I0 = df_indis.I0.values # initial values
IF = df_indis.IF.values # final values
success_rates = df_indis.successRates.values # success rates
R = df_indis.instrumental # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = dict([(code, i) for i, code in enumerate(df_indis.seriesCode)]) # used to build the network matrix

### Interdependency network

In [4]:
df_net = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_network.csv')

A = np.zeros((N, N)) # adjacency matrix
for index, row in df_net.iterrows():
    i = indis_index[row.origin]
    j = indis_index[row.destination]
    w = row.weight
    A[i,j] = w

### Budget

In [5]:
df_exp = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_expenditure.csv')

Bs = df_exp.values[:,1::] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

### Budget-indicator mapping

In [6]:
df_rela = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_relational_table.csv')

B_dict = {} # PPI needs the relational table in the form of a Python dictionary
for index, row in df_rela.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

## Calibrate

Now I run the calibration function to show that it works. Before that, let me explain a couple of new inputs that the user needs to provide:

* <strong>threshold</strong>: How well should the model be fit.
* <strong>parallel_processes</strong>: The number of processes (workers) to be ran in parallel.
* <strong>verbose</strong>: Whether to print or not the outputs as the calibration progresses.
* <strong>low_precision_counts</strong>: The number of iterations that use few Monte Carlo simulations.

The <strong>threshold</strong> parameter indicates the quality of the goodness of fit. More specifically, how good should the worst-fitted indicator be. The best possible fit is close to 1, but cannot be exactly 1 due to the stochasticity of the model. The higher the threshold, the mode Monte Carlo simulations are needed and, thus, the more time and computational resources are needed to complete the calibration.

Parameter <strong>parallel_processes</strong> is used to enhance efficiency. Since each Monte Carlo simulation is independent of each other, this workload can be distributed across multiple cores or processors. Today, most personal devices have the capability of handling this distributed load, so here I show how to calibrate the model using 4 parallel processes. It is recommended that you know how many cores or processors your equipment has, and that <strong>parallel_processes</strong> does not exceed that number. Otherwise, the performance of the calibration may be sub-optimal.

Finally, the <strong>low_precision_counts</strong> parameter helps accelerating the calibration. At the beginning of the calibration, the algorithm proposes a random set of parameters for the model. Because this proposal is unrelated to the true parameters, the errors tend to be large. In the presence of large errors, one can improve the goodness of fit without needing too much precision in each evaluation (i.e., without running too many Monte Carlo simulations). Hence, this parameter determines how many low-precision iterations of the algorithm should be run before proceeding to the high-precision ones. This accelerates the calibration procedure substantially.

In [7]:
T = Bs.shape[1]
parallel_processes = 4 # number of cores to use
threshold = 0.95 # the quality of the calibration (I choose a medium quality for illustration purposes)
low_precision_counts = 100 # number of low-quality iterations to accelerate the calibration

parameters = ppi.calibrate(I0, IF, success_rates, A=A, R=R, qm=qm, rl=rl, Bs=Bs, B_dict=B_dict,
              T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=True,
             low_precision_counts=low_precision_counts)

Iteration: 1 .    Worst goodness of fit: -175734942312.50555
Iteration: 2 .    Worst goodness of fit: -53909464070.38116
Iteration: 3 .    Worst goodness of fit: -13539337307.769817
Iteration: 4 .    Worst goodness of fit: -3351751834.6638684
Iteration: 5 .    Worst goodness of fit: -494821028.4322585
Iteration: 6 .    Worst goodness of fit: -226993243.60603178
Iteration: 7 .    Worst goodness of fit: -53415456.59575788
Iteration: 8 .    Worst goodness of fit: -10988455.398943424
Iteration: 9 .    Worst goodness of fit: -2278726.6615180257
Iteration: 10 .    Worst goodness of fit: -670906.565452804
Iteration: 11 .    Worst goodness of fit: -151887.22355055533
Iteration: 12 .    Worst goodness of fit: -26032.8007320495
Iteration: 13 .    Worst goodness of fit: -10542.658320453575
Iteration: 14 .    Worst goodness of fit: -2073.378641084521
Iteration: 15 .    Worst goodness of fit: -989.0768154794257
Iteration: 16 .    Worst goodness of fit: -389.77180482709736
Iteration: 17 .    Worst g

## Calibration outputs

The output of the calibration function is a matrix with the following columns:

* <strong>alpha</strong>: the parameters related to structural constraints
* <strong>alpha_prime</strong>: the parameters related to structural costs
* <strong>beta</strong>: the parameters related to the probability of success
* <strong>T</strong>: the number of simulation periods
* <strong>error_alpha</strong>: the errors associated to the parameters $\alpha$ and $\alpha'$
* <strong>error_beta</strong>: the errors associated to the parameters $\beta$
* <strong>GoF_alpha</strong>: the goodness-of-fit associated to the parameters $\alpha$ and $\alpha'$
* <strong>GoF_beta</strong>: the goodness-of-fit associated to the parameters $\beta$

The top row of this matrix contains the column names, so I just need to transform these data into a DataFrame to export it.

In [8]:
df_params = pd.DataFrame(parameters[1::], columns=parameters[0])

In [9]:
df_params

,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
0,0.003969130942583395,2.559810733421417e-06,5.496569254751349e-09,60,-0.0021472071642725243,-0.0175942380454156,0.9823245264967766,0.9814797494258783
1,0.00899139231793503,7.346799850504838e-07,1.0521193829791938e-08,nan,0.003634599091273283,0.0007786883894179697,0.984088399595316,0.999180328011139
2,0.010834556324662518,5.207871129234653e-07,6.505799185833022e-09,nan,0.0030843682593520994,0.005680452855669871,0.9902965202421387,0.9940205759414001
3,0.002824612144856482,3.589386210094633e-08,5.219687136605831e-09,nan,-0.0018255632054666737,-0.010746363994844299,0.9823706673414702,0.9886880379001639
4,2.1998634899921643e-05,0.003380988518365714,4.996475221180897e-10,nan,-0.002219580627580653,-0.0008004659026713545,0.9882737005152745,0.9839906819465729
...,...,...,...,...,...,...,...,...
58,0.0012030324828450024,4.0361774820261685e-08,9.710835725068536e-09,nan,-9.561586496897867e-05,-0.0037352849753199413,0.9973077797301405,0.9960681210786105
59,0.012614523450233011,0.0015033483154387845,3.565649219986557,nan,0.002577196010118299,0.0073743073336211,0.9952488289115996,0.9922375712277672
60,0.010212999801629584,1.1942952178728449e-06,7.765220810509205e-09,nan,0.0017116426855541267,-0.003556749661242642,0.9956650348953838,0.9962560529881657
61,0.006635175468262367,3.77257614429579e-07,7.803372341060447e-09,nan,-0.0002014042866755883,0.0007610875702028785,0.9992187904150489,0.9991988551892601


## Save parameters data

In [10]:
df_params.to_csv('parameters.csv', index=False)